# Notebook 01 — Run GUI + Export a Colab Zip

This is the notebook I run **locally** whenever I want to play Minesweeper and collect new states.

What I do here:

- I launch the PyQt GUI (manual play, logic bot, NN tab, saved-state viewer)
- I save game-state JSONs locally under `data/game_states/`
- After I close the GUI, I export a single `.zip` bundle that I upload into Colab

I’m not using Google Drive mounting/sync in this workflow anymore, because it keeps hanging in Colab.



In [1]:
import sys
from pathlib import Path

# Repo root (local): this notebook lives in <repo>/notebooks/
# I just walk upward until I see `minesweeper/` + `models/`.
repo_root = None
for p in [Path.cwd(), *Path.cwd().parents]:
    if (p / 'minesweeper').exists() and (p / 'models').exists():
        repo_root = p
        break

if repo_root is None:
    raise FileNotFoundError('I could not find repo root (expected minesweeper/ + models/)')

sys.path.insert(0, str(repo_root))
print(f"Repo root: {repo_root}")



Repo root: /Users/rachitasaini/Desktop/Rutgers/Fall 2026/Deep Learning 01-198-462/minesweeper-dl-submission


In [2]:
# Zip export config (this is what I upload into Colab)
from pathlib import Path

ZIP_NAME = 'colab_bundle.zip'
ZIP_OUT = repo_root / 'exports' / ZIP_NAME
ZIP_OUT.parent.mkdir(parents=True, exist_ok=True)

# I include the core code + data + writeups.
INCLUDE_DIRS = ['minesweeper', 'models', 'data']
INCLUDE_FILES = ['requirements.txt', 'README.md', 'Minesweeper DL.pdf']

EXCLUDE_DIR_NAMES = {'.git', '.venv', 'venv', '__pycache__', '.ipynb_checkpoints', 'exports'}



In [3]:
from pathlib import Path

# I always save locally while I play (fast + no network dependency).
# After I close the GUI, I bundle `data/` into my Colab zip.
state_dir = repo_root / "data" / "game_states"
state_dir.mkdir(parents=True, exist_ok=True)

print(f"Saving game states to: {state_dir}")



Saving game states to: /Users/rachitasaini/Desktop/Rutgers/Fall 2026/Deep Learning 01-198-462/minesweeper-dl-submission/data/game_states


In [4]:
# Launch the GUI (local only)
import sys

from PyQt5.QtWidgets import QApplication

from minesweeper import MinesweeperGUI, StateCollector

app = QApplication.instance() or QApplication(sys.argv)
collector = StateCollector(output_dir=str(state_dir), auto_save=True)

gui = MinesweeperGUI(collect_states=True, state_collector=collector)
gui.show()
app.exec_()

# StateCollector autosaves as the game runs.
if collector.current_filename:
    print(f"Saved: {collector.current_filename} (states={len(collector.states)})")



2026-01-05 07:46:22.948 Python[31079:18909685] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [5]:
# Toggle: zip export
# I keep this off by default so I don't accidentally generate a zip every time.
EXPORT_ZIP = True


In [6]:
# After I close the GUI, I can export a zip that I upload into Colab.

if not EXPORT_ZIP:
    print('Zip export is disabled (EXPORT_ZIP=False).')
else:
    import zipfile

    zip_path = ZIP_OUT

    if zip_path.exists():
        zip_path.unlink()

    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as z:
        for d in INCLUDE_DIRS:
            root = repo_root / d
            if not root.exists():
                continue
            for p in root.rglob('*'):
                if p.is_dir():
                    continue
                rel = p.relative_to(repo_root)
                if any(part in EXCLUDE_DIR_NAMES for part in rel.parts):
                    continue
                z.write(p, arcname=str(rel))

        for f in INCLUDE_FILES:
            fp = repo_root / f
            if fp.exists() and fp.is_file():
                z.write(fp, arcname=str(Path(f)))

    mb = zip_path.stat().st_size / (1024 * 1024)
    print(f"Wrote: {zip_path} ({mb:.2f} MB)")
    print("In Colab: upload this zip into /content (Files panel -> Upload).")



Wrote: /Users/rachitasaini/Desktop/Rutgers/Fall 2026/Deep Learning 01-198-462/minesweeper-dl-submission/exports/colab_bundle.zip (76.80 MB)
In Colab: upload this zip into /content (Files panel -> Upload).
